In [3]:
%cd ..

c:\Users\HP\OneDrive - University of Moratuwa\Desktop\E-Vision-Projects\Shelf_Product_Count_Generation


In [4]:
from dotenv import load_dotenv
import os

_ = load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')
antropic_api_key = os.getenv("ANTHROPIC_API_KEY")

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
            model='gpt-4o-mini',
            api_key=os.getenv("OPENAI_API_KEY")
        )

llm.invoke('Hi')

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_11f3029f6b', 'id': 'chatcmpl-ClVBi5hxBhVRNL4BSi1hyltOfJl03', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--3e8e57dd-23cd-4b33-8683-478120114972-0', usage_metadata={'input_tokens': 8, 'output_tokens': 9, 'total_tokens': 17, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [6]:
from ultralytics import YOLO 
import numpy as np 

class ShelfLabelDetector:
    def __init__(self, yolo_model_path='data/best.pt'):
        self.yolo_model = YOLO(yolo_model_path)
        
    def detect_shelf_labels(self, image: np.ndarray, confidence_threshold: float = 0.25):
        
        results = self.yolo_model(image, conf=confidence_threshold, verbose=False)
        
        processed_detections = []
        
        # Process YOLO results
        for result in results:
            boxes = result.boxes
            for box in boxes:
                # Get bounding box coordinates (xyxy format)
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                confidence = float(box.conf[0])
                class_id = int(box.cls[0])
                class_name = result.names[class_id]
                
                # Accept all detections from custom trained model
                processed_detections.append({
                    'bbox': [int(x1), int(y1), int(x2), int(y2)],
                    'confidence': confidence,
                    'class': class_name
                })
        
        print(f"YOLO detected {len(processed_detections)} products")
        return processed_detections

In [7]:
import cv2

image = cv2.imread('data/test_images/IMG_2329.jpeg')
detector = ShelfLabelDetector()
processed_detections = detector.detect_shelf_labels(image)

YOLO detected 26 products


In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
import base64
import numpy as np

# Define Pydantic model for structured output
class Product(BaseModel):
    brand: str = Field(description="Extract the exact brand name from visible labels or logos")
    product_name: str = Field(description="Full product name or title as shown on the packaging")
    variant: str = Field(description="Identify the variant or type")
    primary_colors: list[str] = Field(description="List the dominant colors of the packaging")
    packaging_type: str = Field(description="Type of container (e.g., 'Bottle', 'Tube', 'Jar', 'Box', 'Can')")
    shape: str = Field(description="Describe the container shape (e.g., 'Cylindrical', 'Rectangular', 'Oval')")
    size_volume: str = Field(description="Extract exact size/volume from label (e.g., '200ml', '500g') or estimate if not visible")
    logo_description: str = Field(description="Brief description of the brand logo appearance and position.")
    key_text: list[str] = Field(description="Extract main visible text or claims on the label")
    distinctive_elements: list[str] = Field(description="Any unique design features, patterns, or color combinations that make this product recognizable")

class ProductInfo(BaseModel):
    product: Product

class ProductInfoExtractor:
    def __init__(self):
        pass
    
    def _encode_image_to_base64(self, image: np.ndarray) -> str:
        """Convert numpy image to base64 string."""
        _, buffer = cv2.imencode('.jpg', image)
        return base64.b64encode(buffer).decode('utf-8')
    
    def extract_text_from_product(self, product: np.array):
        
        base64_image = self._encode_image_to_base64(product)
        
        parser = JsonOutputParser()
        
        prompt = """
        Analyze the product image and extract key identifying features.
        
        {format_instructions}
        
        Extract: brand, product_name, variant, primary_colors (list), packaging_type, 
        shape, size_volume, logo_description, key_text (list), distinctive_elements (list).
        
        Mark unclear features as "Unknown".
        """
        
        format_instructions = parser.get_format_instructions()
        prompt = prompt.format(format_instructions=format_instructions)
        
        message = HumanMessage(
            content=[
                {"type": "text", "text": prompt},
                {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}},
                ]
            )
        
        response = llm.invoke([message])
        
        result = parser.parse(response.content)
        
        return result

In [2]:
import cv2
image = cv2.imread('data/test_images/cropped_image_IMG_2320_9.jpg')
product_info =ProductInfoExtractor()
result = product_info.extract_text_from_product(product=image)

error: OpenCV(4.12.0) D:\a\opencv-python\opencv-python\opencv\modules\imgcodecs\src\loadsave.cpp:1596: error: (-215:Assertion failed) !_img.empty() in function 'cv::imencodeWithMetadata'


In [10]:
result

{'brand': 'Janet',
 'product_name': 'Hair Oil',
 'variant': 'Dark Henna',
 'primary_colors': ['Red', 'Green', 'Pink'],
 'packaging_type': 'Box',
 'shape': 'Rectangular',
 'size_volume': 'Unknown',
 'logo_description': "Stylized text of 'Janet' with 'Ayurveda' beneath it",
 'key_text': ['HAIR FALL CONTROL', 'DARK HENNA', 'HAIR OIL', 'PARABEN FREE'],
 'distinctive_elements': ['Images of plants',
  'Colorful squares with plant images']}

In [11]:
import json
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field

class MatchResult(BaseModel):
    is_match: bool = Field(description="Whether a match was found in the reference database")
    matched_product_id: str = Field(description="ID of the matched product from reference database, or 'None' if no match")
    confidence_score: float = Field(description="Confidence score of the match (0.0 to 1.0)")
    reasoning: str = Field(description="Explanation of why this product was matched or not matched")

class ProductMatcherLLM:
    def __init__(self, reference_db_path='data/product_reference_database.json'):
        self.reference_db_path = reference_db_path
        self.reference_products = self._load_reference_database()
    
    def _load_reference_database(self):
        """Load the reference product database."""
        try:
            with open(self.reference_db_path, 'r') as f:
                return json.load(f)
        except FileNotFoundError:
            print(f"Warning: Reference database not found at {self.reference_db_path}")
            return {}
    
    def match_product(self, detected_product_info: dict) -> dict:
        """
        Match detected product with reference database using LLM.
        
        Args:
            detected_product_info: Dictionary containing extracted product information
            
        Returns:
            Dictionary with match results including is_match, matched_product_id, confidence_score, reasoning
        """
        parser = JsonOutputParser(pydantic_object=MatchResult)
        
        prompt = f"""
        You are a product matching expert. Compare the detected product with the reference database and find the best match.
        
        **Detected Product Information:**
        {json.dumps(detected_product_info, indent=2)}
        
        **Reference Product Database:**
        {json.dumps(self.reference_products, indent=2)}
        
        **Task:**
        1. Carefully compare the detected product features (brand, product_name, variant, colors, packaging, etc.) with each product in the reference database
        2. Consider partial matches and variations in naming
        3. If a strong match is found (>70% similarity), set is_match to true and provide the product ID
        4. If no confident match is found, set is_match to false and matched_product_id to "None"
        5. Provide a confidence score between 0.0 and 1.0
        6. Explain your reasoning clearly
        
        {parser.get_format_instructions()}
        
        Return the result as JSON with: is_match, matched_product_id, confidence_score, reasoning
        """
        
        message = HumanMessage(content=prompt)
        response = llm.invoke([message])
        result = parser.parse(response.content)
        
        return result
    
    def batch_match_products(self, detected_products: list[dict]) -> list[dict]:
        """
        Match multiple detected products with reference database.
        
        Args:
            detected_products: List of detected product information dictionaries
            
        Returns:
            List of match results for each product
        """
        results = []
        for product in detected_products:
            match_result = self.match_product(product)
            results.append({
                'detected_product': product,
                'match_result': match_result
            })
        return results

In [12]:
# Initialize matcher
matcher = ProductMatcherLLM(reference_db_path='data/product_reference_database.json')

# Match a single product
match_result = matcher.match_product(result)
print(match_result)

{'is_match': True, 'matched_product_id': '1009', 'confidence_score': 0.95, 'reasoning': "The detected product is from the brand 'Janet', with the product name 'Hair Oil' and variant 'Dark Henna'. The primary colors match exactly with 'Red', 'Green', and 'Pink', and the packaging type is also 'Box' with a rectangular shape. The key text features 'HAIR FALL CONTROL', 'DARK HENNA', and 'PARABEN FREE', which all correspond to the matched product. The distinctive elements like 'Images of plants' and 'Color blocks' are also present in the match. Given these strong similarities across multiple attributes, the confidence score reflects a very high likelihood of being a true match."}


In [13]:
import json
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from rapidfuzz import fuzz, process

class MatchResult(BaseModel):
    is_match: bool = Field(description="Whether a match was found in the reference database")
    matched_product_id: str = Field(description="ID of the matched product from reference database, or 'None' if no match")
    confidence_score: float = Field(description="Confidence score of the match (0.0 to 1.0)")
    reasoning: str = Field(description="Explanation of why this product was matched or not matched")

class ProductMatcherLLM:
    def __init__(self, reference_db_path='data/product_reference_database.json', top_k_candidates=3, similarity_threshold=60):
        """
        Initialize product matcher with two-stage matching: fuzzy filter + LLM verification.
        
        Args:
            reference_db_path: Path to reference product database JSON
            top_k_candidates: Number of top candidates to send to LLM (default: 3)
            similarity_threshold: Minimum similarity score for candidates (0-100, default: 60)
        """
        self.reference_db_path = reference_db_path
        self.reference_products = self._load_reference_database()
        self.top_k_candidates = top_k_candidates
        self.similarity_threshold = similarity_threshold
    
    def _load_reference_database(self):
        """Load the reference product database."""
        try:
            with open(self.reference_db_path, 'r') as f:
                return json.load(f)
        except FileNotFoundError:
            print(f"Warning: Reference database not found at {self.reference_db_path}")
            return {}
    
    def _calculate_composite_similarity(self, detected: dict, reference: dict) -> float:
        """
        Calculate composite similarity score using multiple attributes.
        
        Args:
            detected: Detected product info
            reference: Reference product info
            
        Returns:
            Weighted similarity score (0-100)
        """
        scores = []
        weights = []
        
        # Brand matching (weight: 0.35)
        if detected.get('brand') and reference.get('brand'):
            brand_score = fuzz.token_sort_ratio(
                detected['brand'].lower(), 
                reference['brand'].lower()
            )
            scores.append(brand_score)
            weights.append(0.35)
        
        # Product name matching (weight: 0.35)
        if detected.get('product_name') and reference.get('product_name'):
            name_score = fuzz.token_sort_ratio(
                detected['product_name'].lower(),
                reference['product_name'].lower()
            )
            scores.append(name_score)
            weights.append(0.35)
        
        # Color matching (weight: 0.15)
        detected_colors = [c.lower() for c in detected.get('primary_colors', [])]
        ref_colors = [c.lower() for c in reference.get('primary_colors', [])]
        if detected_colors and ref_colors:
            color_overlap = len(set(detected_colors) & set(ref_colors))
            color_score = (color_overlap / max(len(detected_colors), len(ref_colors))) * 100
            scores.append(color_score)
            weights.append(0.15)
        
        # Packaging type matching (weight: 0.10)
        if detected.get('packaging_type') and reference.get('packaging_type'):
            packaging_score = fuzz.ratio(
                detected['packaging_type'].lower(),
                reference['packaging_type'].lower()
            )
            scores.append(packaging_score)
            weights.append(0.10)
        
        # Size/volume matching (weight: 0.05)
        if detected.get('size_volume') and reference.get('size_volume'):
            size_score = fuzz.partial_ratio(
                detected['size_volume'].lower(),
                reference['size_volume'].lower()
            )
            scores.append(size_score)
            weights.append(0.05)
        
        # Calculate weighted average
        if not scores:
            return 0.0
        
        total_weight = sum(weights)
        weighted_score = sum(s * w for s, w in zip(scores, weights)) / total_weight
        
        return weighted_score
    
    def _filter_candidate_products(self, detected_product_info: dict) -> list:
        """
        Filter top-k candidate products using RapidFuzz similarity matching.
        
        Args:
            detected_product_info: Extracted product information
            
        Returns:
            List of top candidates with scores
        """
        candidates = []
        
        for product_id, product_data in self.reference_products.items():
            similarity_score = self._calculate_composite_similarity(
                detected_product_info, 
                product_data
            )
            
            # Only consider candidates above threshold
            if similarity_score >= self.similarity_threshold:
                candidates.append({
                    'product_id': product_id,
                    'product_data': product_data,
                    'similarity_score': similarity_score
                })
        
        # Sort by similarity score (descending) and take top-k
        candidates.sort(key=lambda x: x['similarity_score'], reverse=True)
        top_candidates = candidates[:self.top_k_candidates]
        
        print(f"\n--- Candidate Filtering Results ---")
        print(f"Total reference products: {len(self.reference_products)}")
        print(f"Products above threshold ({self.similarity_threshold}%): {len(candidates)}")
        print(f"Top {self.top_k_candidates} candidates for LLM verification:")
        
        for i, candidate in enumerate(top_candidates, 1):
            print(f"  {i}. [{candidate['product_id']}] - Similarity: {candidate['similarity_score']:.1f}%")
            print(f"     Brand: {candidate['product_data'].get('brand', 'N/A')}")
            print(f"     Product: {candidate['product_data'].get('product_name', 'N/A')}")
        
        return top_candidates
    
    def match_product(self, detected_product_info: dict) -> dict:
        """
        Match detected product using two-stage approach:
        1. RapidFuzz filtering to get top-k candidates (fast, cheap)
        2. LLM verification for final match decision (accurate, expensive)
        
        Args:
            detected_product_info: Dictionary containing extracted product information
            
        Returns:
            Dictionary with match results
        """
        # Stage 1: Fast filtering with RapidFuzz
        candidates = self._filter_candidate_products(detected_product_info)
        
        if not candidates:
            print("No candidates found above similarity threshold")
            return {
                'is_match': False,
                'matched_product_id': 'None',
                'confidence_score': 0.0,
                'reasoning': f'No candidate products found with similarity >= {self.similarity_threshold}%'
            }
        
        # If only one high-confidence candidate (>90%), skip LLM
        if len(candidates) == 1 and candidates[0]['similarity_score'] > 90:
            print(f"High confidence match found: {candidates[0]['product_id']} ({candidates[0]['similarity_score']:.1f}%)")
            return {
                'is_match': True,
                'matched_product_id': candidates[0]['product_id'],
                'confidence_score': candidates[0]['similarity_score'] / 100,
                'reasoning': f"High similarity match ({candidates[0]['similarity_score']:.1f}%) - LLM verification skipped"
            }
        
        # Stage 2: LLM verification for ambiguous cases
        print(f"\nSending {len(candidates)} candidates to LLM for final verification...")
        
        filtered_ref_db = {
            c['product_id']: c['product_data'] 
            for c in candidates
        }
        
        parser = JsonOutputParser(pydantic_object=MatchResult)
        
        prompt = f"""
        You are a product matching expert. Compare the detected product with pre-filtered candidate products.
        
        **Detected Product Information:**
        {json.dumps(detected_product_info, indent=2)}
        
        **Top Candidate Products (pre-filtered by similarity scores):**
        {json.dumps(filtered_ref_db, indent=2)}
        
        **Pre-filter Similarity Scores:**
        {json.dumps({c['product_id']: f"{c['similarity_score']:.1f}%" for c in candidates}, indent=2)}
        
        **Instructions:**
        1. These candidates were pre-selected based on brand, name, color, and packaging similarity
        2. Compare ALL product attributes: brand, product_name, variant, colors, packaging, size, distinctive elements
        3. Consider that product names may vary slightly (abbreviations, word order, etc.)
        4. If a confident match exists (>75% certainty), set is_match=true with the product_id
        5. If candidates are too different or ambiguous, set is_match=false
        6. Provide confidence_score (0.0-1.0) and detailed reasoning
        
        {parser.get_format_instructions()}
        """
        
        message = HumanMessage(content=prompt)
        response = llm.invoke([message])
        result = parser.parse(response.content)
        
        print(f"\nLLM Decision: {'MATCH' if result['is_match'] else 'NO MATCH'}")
        if result['is_match']:
            print(f"Matched Product: {result['matched_product_id']}")
        print(f"Confidence: {result['confidence_score']:.2f}")
        print(f"Reasoning: {result['reasoning']}")
        
        return result
    
    def batch_match_products(self, detected_products: list[dict]) -> list[dict]:
        """
        Match multiple detected products efficiently.
        
        Args:
            detected_products: List of detected product information dictionaries
            
        Returns:
            List of match results for each product
        """
        results = []
        print(f"\n{'='*60}")
        print(f"BATCH MATCHING: {len(detected_products)} products")
        print(f"{'='*60}")
        
        for i, product in enumerate(detected_products, 1):
            print(f"\n{'─'*60}")
            print(f"Product {i}/{len(detected_products)}")
            print(f"{'─'*60}")
            
            match_result = self.match_product(product)
            results.append({
                'detected_product': product,
                'match_result': match_result
            })
        
        # Summary
        print(f"\n{'='*60}")
        print(f"MATCHING SUMMARY")
        print(f"{'='*60}")
        matched = sum(1 for r in results if r['match_result']['is_match'])
        print(f"Total products: {len(results)}")
        print(f"Matched: {matched}")
        print(f"Not matched: {len(results) - matched}")
        
        return results

In [14]:
matcher = ProductMatcherLLM(
    reference_db_path='data/product_reference_database.json',
    top_k_candidates=10  # Adjust based on your needs
)

# Match a single product
match_result = matcher.match_product(result)
print(match_result)


--- Candidate Filtering Results ---
Total reference products: 21
Products above threshold (60%): 3
Top 10 candidates for LLM verification:
  1. [1009] - Similarity: 100.0%
     Brand: Janet
     Product: Hair Oil
  2. [1020] - Similarity: 90.0%
     Brand: Janet
     Product: Hair Oil
  3. [1010] - Similarity: 88.8%
     Brand: Janet
     Product: Hair Oil

Sending 3 candidates to LLM for final verification...

LLM Decision: MATCH
Matched Product: 1009
Confidence: 0.95
Reasoning: The detected product matches the candidate product with ID 1009 on several critical attributes: the brand is 'Janet', the product name is 'Hair Oil', and the variant is 'Dark Henna'. The primary colors are identical (Red, Green, Pink), packaging type and shape (both Box and Rectangular) match, and the size/volume is listed as 'Unknown'. The logo descriptions are slightly different but retain the core visual identity. The key texts are nearly identical, although one is missing 'HAIR OIL', which is understandab

In [15]:
import cv2
import numpy as np
from typing import List, Dict
from collections import Counter

class ProductCounter:
    def __init__(self, detector: ShelfLabelDetector, info_extractor: ProductInfoExtractor, matcher: ProductMatcherLLM):
        """
        Initialize product counter with detector, info extractor, and matcher.
        
        Args:
            detector: ShelfLabelDetector instance for detecting products
            info_extractor: ProductInfoExtractor instance for extracting product info
            matcher: ProductMatcherLLM instance for matching with reference database
        """
        self.detector = detector
        self.info_extractor = info_extractor
        self.matcher = matcher
        
    def _crop_detection(self, image: np.ndarray, bbox: List[int]) -> np.ndarray:
        """
        Crop detected region from image.
        
        Args:
            image: Full shelf image
            bbox: Bounding box [x1, y1, x2, y2]
            
        Returns:
            Cropped image region
        """
        x1, y1, x2, y2 = bbox
        return image[y1:y2, x1:x2]
    
    def count_products(self, image: np.ndarray, confidence_threshold: float = 0.25) -> Dict:
        """
        Count products in shelf image.
        
        Args:
            image: Shelf image (numpy array)
            confidence_threshold: Minimum confidence for detection
            
        Returns:
            Dictionary with product counts and detailed results
        """
        # Step 1: Detect products
        print("Step 1: Detecting products with YOLO...")
        detections = self.detector.detect_shelf_labels(image, confidence_threshold)
        
        if not detections:
            print("No products detected!")
            return {
                'total_detected': 0,
                'product_counts': {},
                'unmatched_count': 0,
                'detailed_results': []
            }
        
        # Step 2: Extract info and match each detection
        print(f"\nStep 2: Extracting info and matching {len(detections)} detected products...")
        detailed_results = []
        matched_products = []
        unmatched_count = 0
        
        for i, detection in enumerate(detections, 1):
            print(f"\n{'='*60}")
            print(f"Processing detection {i}/{len(detections)}")
            print(f"{'='*60}")
            
            # Crop detection
            bbox = detection['bbox']
            cropped_image = self._crop_detection(image, bbox)
            
            # Extract product info
            print("Extracting product information...")
            try:
                product_info = self.info_extractor.extract_text_from_product(cropped_image)
            except Exception as e:
                print(f"Error extracting product info: {e}")
                unmatched_count += 1
                detailed_results.append({
                    'detection_id': i,
                    'bbox': bbox,
                    'yolo_confidence': detection['confidence'],
                    'product_info': None,
                    'match_result': None,
                    'status': 'extraction_failed'
                })
                continue
            
            # Match with reference database
            print("Matching with reference database...")
            try:
                match_result = self.matcher.match_product(product_info)
            except Exception as e:
                print(f"Error matching product: {e}")
                unmatched_count += 1
                detailed_results.append({
                    'detection_id': i,
                    'bbox': bbox,
                    'yolo_confidence': detection['confidence'],
                    'product_info': product_info,
                    'match_result': None,
                    'status': 'matching_failed'
                })
                continue
            
            # Store results
            if match_result['is_match']:
                matched_products.append(match_result['matched_product_id'])
                status = 'matched'
            else:
                unmatched_count += 1
                status = 'unmatched'
            
            detailed_results.append({
                'detection_id': i,
                'bbox': bbox,
                'yolo_confidence': detection['confidence'],
                'product_info': product_info,
                'match_result': match_result,
                'status': status
            })
        
        # Step 3: Count products
        print(f"\n{'='*60}")
        print("Step 3: Counting products...")
        print(f"{'='*60}")
        
        product_counts = Counter(matched_products)
        
        return {
            'total_detected': len(detections),
            'total_matched': len(matched_products),
            'unmatched_count': unmatched_count,
            'product_counts': dict(product_counts),
            'detailed_results': detailed_results
        }
    
    def visualize_results(self, image: np.ndarray, count_results: Dict, output_path: str = None) -> np.ndarray:
        """
        Visualize counting results on image.
        
        Args:
            image: Original shelf image
            count_results: Results from count_products()
            output_path: Optional path to save visualization
            
        Returns:
            Annotated image
        """
        img_copy = image.copy()
        
        for result in count_results['detailed_results']:
            bbox = result['bbox']
            x1, y1, x2, y2 = bbox
            
            # Color based on status
            if result['status'] == 'matched':
                color = (0, 255, 0)  # Green for matched
                label = result['match_result']['matched_product_id']
            else:
                color = (0, 0, 255)  # Red for unmatched
                label = 'Unknown'
            
            # Draw bounding box
            cv2.rectangle(img_copy, (x1, y1), (x2, y2), color, 2)
            
            # Draw label
            label_text = f"{result['detection_id']}: {label}"
            cv2.putText(img_copy, label_text, (x1, y1 - 10),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        
        if output_path:
            os.makedirs(os.path.dirname(output_path), exist_ok=True)
            cv2.imwrite(output_path, img_copy)
            print(f"Visualization saved to: {output_path}")
        
        return img_copy
    
    def print_summary(self, count_results: Dict):
        """
        Print a formatted summary of counting results.
        
        Args:
            count_results: Results from count_products()
        """
        print(f"\n{'='*60}")
        print("PRODUCT COUNTING SUMMARY")
        print(f"{'='*60}")
        print(f"Total products detected: {count_results['total_detected']}")
        print(f"Successfully matched: {count_results['total_matched']}")
        print(f"Unmatched/Failed: {count_results['unmatched_count']}")
        
        print(f"\n{'─'*60}")
        print("Product Counts:")
        print(f"{'─'*60}")
        
        if count_results['product_counts']:
            for product_id, count in sorted(count_results['product_counts'].items()):
                print(f"  {product_id}: {count}")
        else:
            print("  No matched products")
        
        print(f"{'='*60}\n")

In [17]:
# Initialize components
detector = ShelfLabelDetector(yolo_model_path='data/best.pt')
info_extractor = ProductInfoExtractor()
matcher = ProductMatcherLLM(
    reference_db_path='data/product_reference_database.json',
    top_k_candidates=10
)

# Create counter
counter = ProductCounter(detector, info_extractor, matcher)

# Load image
image = cv2.imread('data/test_images/IMG_2230.jpeg')

# Count products
results = counter.count_products(image, confidence_threshold=0.25)

# Print summary
counter.print_summary(results)

# Visualize results
annotated_image = counter.visualize_results(image, results, output_path='data/output/counted_products.jpg')

# Display
cv2.imshow('Product Counts', annotated_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

Step 1: Detecting products with YOLO...
YOLO detected 34 products

Step 2: Extracting info and matching 34 detected products...

Processing detection 1/34
Extracting product information...
Matching with reference database...

--- Candidate Filtering Results ---
Total reference products: 21
Products above threshold (60%): 0
Top 10 candidates for LLM verification:
No candidates found above similarity threshold

Processing detection 2/34
Extracting product information...
Matching with reference database...

--- Candidate Filtering Results ---
Total reference products: 21
Products above threshold (60%): 6
Top 10 candidates for LLM verification:
  1. [1012] - Similarity: 92.5%
     Brand: emami
     Product: 7 OILS IN ONE
  2. [1004] - Similarity: 90.0%
     Brand: Emami
     Product: 7 Oils in One
  3. [1005] - Similarity: 87.5%
     Brand: emami
     Product: 7 OILS IN ONE
  4. [1000] - Similarity: 85.0%
     Brand: emami
     Product: 7 OILS IN ONE
  5. [1001] - Similarity: 81.0%
     Br